<a href="https://colab.research.google.com/github/LeyloJohnny/First-ML/blob/main/Random_3_or_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 5.9MB/s 
     |████████████████████████████████| 1.2MB 10.8MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
     |████████████████████████████████| 194kB 30.0MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 12.8MB 260kB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

#Make model to differentiate 3 from 7

In [3]:
#download a sample of MNIST that contains images of 3 and 7 digits
path = untar_data(URLs.MNIST_SAMPLE)

In [4]:
#hide
Path.BASE_PATH = path

In [6]:
#sort dataset to make sure we get the same order of files
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()

(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png')...]

#Define model

In [8]:
#make tensors for all threes and all sevens
seven_tensors = [tensor(Image.open(o)) for o in sevens]
three_tensors = [tensor(Image.open(o)) for o in threes]
len(three_tensors),len(seven_tensors)

(6131, 6265)

In [9]:
#stack images --> make a 3D tensor from all tensors to get pixel average afterwards
#divide by 255 to get numbers between 0 and 1
stacked_sevens = torch.stack(seven_tensors).float()/255
stacked_threes = torch.stack(three_tensors).float()/255
stacked_threes.shape

torch.Size([6131, 28, 28])

In [15]:
#create tensors from validation "valid" dataset
valid_3_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()])
valid_3_tens = valid_3_tens.float()/255
valid_7_tens = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()])
valid_7_tens = valid_7_tens.float()/255

#check if the shape is correct
valid_3_tens.shape,valid_7_tens.shape

(torch.Size([1010, 28, 28]), torch.Size([1028, 28, 28]))

In [11]:
#define training set and convert 3-rank tensor into 2-rank tensor
train_x = torch.cat([stacked_threes, stacked_sevens]).view(-1, 28*28)
train_x.shape

torch.Size([12396, 784])

In [12]:
#label data, 3s are given the label 1--> define as train_y
train_y = tensor([1]*len(threes) + [0]*len(sevens)).unsqueeze(1)
train_y.shape

torch.Size([12396, 1])

In [13]:
#combine values to build a dataset consisting of a list of tuples (x, y) using zip
dset = list(zip(train_x,train_y))
x,y = dset[0]
x.shape,y

(torch.Size([784]), tensor([1]))

In [21]:
#define the validation set similarly to the training set
valid_x = torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tens) + [0]*len(valid_7_tens)).unsqueeze(1)
valid_dset = list(zip(valid_x,valid_y))

In [22]:
#function to initialize random parameters, important: keep track of gradients!
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

In [30]:
#initialiize weights for every pixel
weights = init_params((28*28,1))

In neural networks, the w in the equation y=w*x+b is called the weights, and the b is called the bias. Together, the weights and bias make up the parameters.

In [43]:
#define the bias b, just randomly initialized using our function
bias = init_params(1)

In [44]:
#this way we can calculate a prediction from this model
(train_x[0]*weights.T).sum() + bias

tensor([24.4768], grad_fn=<AddBackward0>)

In [45]:
#now let's do it for all training data using matrix multiplication 
#the matrix multiplication function is called linear1 here
def linear1(xb): return xb@weights + bias
preds = linear1(train_x)
preds

tensor([[24.4768],
        [22.0705],
        [18.5282],
        ...,
        [-3.1462],
        [ 5.8236],
        [ 7.9015]], grad_fn=<AddBackward0>)

In [46]:
#test the accuracy of the model
#predictions > 0.5 = 3
#preds > 0.5 gives True/False --> converted into a float --> compared to labels from train_y
corrects = (preds>0.5).float() == train_y
corrects

tensor([[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [False],
        [False]])

In [47]:
#accuracy by taking the mean of all corrects items
corrects.float().mean().item()

0.5496127605438232

In [49]:
#update weights using loss function 
def mnist_loss(predictions, targets):
  #apply sigmoid function to ensure that values are between 0 and 1
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()